# Intelligent Systems Assignment 1

## Masterball solver

-----------------------------------------

**Name:** Alexander Gonzalez Castañeda
 
**ID:** 1032452063

-----------------------------------------

**Name:** Camilo Andrés Rodriguez

**ID:** 1018446221


### 1. Create a class to model the Masterball problem

A Masterball must be represented as an array of arrays with integer values representing the color of the tile in each position:

A solved masterball must look like this:

```python
[ [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7]
]
```

#### Variables modeling the actions

In [109]:
'''
This variables MUST not be changed.
They represent the movements of the masterball.
'''
R_0 = "Right 0"
R_1 = "Right 1"
R_2 = "Right 2"
R_3 = "Right 3"

V_0 = "Vertical 0"
V_1 = "Vertical 1"
V_2 = "Vertical 2"
V_3 = "Vertical 3"
V_4 = "Vertical 4"
V_5 = "Vertical 5"
V_6 = "Vertical 6"
V_7 = "Vertical 7"


`R_i` moves the `i`th row to the right. For instance, `R_2` applied to the solved state will produce:

```python
[ [0, 1, 2, 3, 4, 5, 6, 7],
  [0, 1, 2, 3, 4, 5, 6, 7],
  [7, 0, 1, 2, 3, 4, 5, 6],
  [0, 1, 2, 3, 4, 5, 6, 7]
]
```

`V_i` performs a clockwise vertical move starting with the `i`th column

`V_1` applied to the above state will produce:

```python
[ [0, 4, 3, 2, 1, 5, 6, 7],
  [0, 3, 2, 1, 0, 5, 6, 7],
  [7, 4, 3, 2, 1, 4, 5, 6],
  [0, 4, 3, 2, 1, 5, 6, 7]
]
```

#### The Masterball problem class

In [110]:
import search
import util
import copy
from random import randint

class MasterballProblem(search.SearchProblem):    
    
    def __init__(self, startState):
        '''
        Store the initial state in the problem representation and any useful
        data.
        Here are some examples of initial states:
        [[0, 1, 4, 5, 6, 2, 3, 7], [0, 1, 3, 4, 5, 6, 3, 7], [1, 2, 4, 5, 6, 2, 7, 0], [0, 1, 4, 5, 6, 2, 3, 7]]
        [[0, 7, 4, 5, 1, 6, 2, 3], [0, 7, 4, 5, 0, 5, 2, 3], [7, 6, 3, 4, 1, 6, 1, 2], [0, 7, 4, 5, 1, 6, 2, 3]]
        [[0, 1, 6, 4, 5, 2, 3, 7], [0, 2, 6, 5, 1, 3, 4, 7], [0, 2, 6, 5, 1, 3, 4, 7], [0, 5, 6, 4, 1, 2, 3, 7]]
        '''
        self.startState = startState.__str__()
        self.expanded = 0
    
    def reset(self):
        self.expanded=0
    
    def setStartState(self, state):
        self.startState = state
    
    def readState(self, string):
        state=[]
        lists=string.split('], [')
        for i in lists:
            a=i.replace('[', '')
            a=a.replace(']', '')
            a=a.replace(' ', '')
            items=a.split(',')
            temp=[]
            for j in items:
                temp.append(int(j))
            state.append(temp)
        return state
    
    
    def makeMove(self, state, action):
        newState = self.readState(state)
        act=action.split()
        index=int(act[1])
        if act[0]=='Right':
            newState[index]=newState[index][-1:]+newState[index][:-1]
        elif act[0]=='Right-':
            newState[index]=newState[index][1:]+newState[index][:1]
        else:
            transp=map(list, zip(*newState))
            temp=[]
            for i in range(0,4):
                j=(index+i)%8
                temp.append(transp[j])
                temp[i].reverse()
            temp.reverse()
            for i in range(0,4):
                j=(index+i)%8
                transp[j]=temp[i]
            newState=map(list, zip(*transp))
        return newState.__str__()
            
    
    def isGoalState(self, state):
        '''
        Define when a given state is a goal state (A correctly colored masterball)
        '''
        s = self.readState(state)
        transp=map(list, zip(*s))
        for idx, latitude in enumerate(transp):
            for item in latitude:
                if idx != item: return False
        return True
    
    def getStartState(self):
        '''
        Implement a method that returns the start state according to the SearchProblem
        contract.
        '''
        return self.startState

    def getSuccessors(self, state):
        '''
        Implement a successor function: Given a state from the masterball
        return a list of the successors and their corresponding actions. 

        This method *must* return a list where each element is a tuple of 
        three elements with the state of the masterball in the first position,
        the action (according to the definition above) in the second position, 
        and the cost of the action in the last position. 

        Note that you should not modify the state.
        '''
        self.expanded += 1
        successors=[]
        for i in range(0, 4):
            action='Right '+str(i)
            successors.append( (self.makeMove(state, action), action, 8) )
        for i in range(0, 8):
            action='Vertical '+str(i)
            successors.append( (self.makeMove(state, action), action, 16) )
        return successors

### 2. Implement iterative deepening search

Follow the example code provided in class and implement iterative deepening search (IDS).

In [111]:
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

def iterativeDeepeningSearch(problem):
    lim=50
    for i in range(1, lim+1):
        visited = {}
        state = problem.getStartState()
        visited[state] = 'gray'
        frontier = util.Stack()
        frontier.push((state, []))
        while not frontier.isEmpty():
            u, actions = frontier.pop()
            if len(actions)>=i: continue
            for v, action, cost in problem.getSuccessors(u):
                if not v in visited:
                    if problem.isGoalState(v):
                        return  actions + [action]
                    visited[v] = 'gray'
                    frontier.push((v, actions + [action]))
            visited[u] = 'green'
    return []

def aStarSearch(problem, heuristic):
    lim=50
    for i in range(1, lim+1):
        visited = {}
        state = problem.getStartState()
        visited[state] = 0
        frontier = util.PriorityQueue()
        frontier.push((state, [], 0),0)
        while not frontier.isEmpty():
            item=frontier.pop()
            u, actions, c = item
            if len(actions)>=i: continue
            for v, action, cost in problem.getSuccessors(u):
                if v in visited and c+cost>=visited[v]: continue
                if problem.isGoalState(v):
                    return  actions + [action]
                visited[v] = c+cost
                frontier.push((v, actions + [action], c+cost), c+cost+heuristic(problem.readState(v)))
    return []



Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


Evaluate it to see what is the maximum depth that it could explore in a reasonable time. Report the results. 

### 3. Implement different heuristics for the problem

Implement at least two admissible and consistent heuristics. Compare A* using the heuristics against IDS calculating the number of expanded nodes and the effective branching factor, in the same way as it is done in figure 3.29 of [Russell10].

In [112]:
def myHeuristic1(state):
    transp=map(list, zip(*state))
    count=0
    for idx, latitude in enumerate(transp):
        for item in latitude:
            if idx == item: count+=1
    return 32-count

def myHeuristic2(state):
    solved = 0
    for index in xrange(0,8):
        if state[0][index] == state[1][index] == state[2][index] == state[3][index]:
            solved += 1
    return 8 - solved

In [113]:
def solveMasterBall(problem, search_function):
    '''
    This function receives a Masterball problem instance and a 
    search_function (IDS or A*S) and must return a list of actions that solve the problem.
    '''
    return search_function

def getBranchFactor(N, depth):
    array=[1]*(depth)
    array.append(-N)
    return np.roots(array).real[depth-1]


problem = MasterballProblem([ [0, 4, 3, 2, 1, 5, 6, 7],
                              [0, 3, 2, 1, 0, 5, 6, 7],
                              [7, 4, 3, 2, 1, 4, 5, 6],
                              [0, 4, 3, 2, 1, 5, 6, 7]])

print '-- IDS --'
solution=solveMasterBall(problem, iterativeDeepeningSearch(problem))
print solution
bf=round(getBranchFactor(problem.expanded, len(solution)),2)
print 'Nodes expanded:', problem.expanded, '  Branching Factor: ', bf
problem.reset()
print '\n-- A*(h1) --'
solution=solveMasterBall(problem, aStarSearch(problem, myHeuristic1))
print solution
bf=round(getBranchFactor(problem.expanded, len(solution)),2)
print 'Nodes expanded:', problem.expanded, '  Branching Factor: ', bf
problem.reset()
print '\n-- A*(h2) --'
solution=solveMasterBall(problem, aStarSearch(problem, myHeuristic2))
print solution
bf=round(getBranchFactor(problem.expanded, len(solution)),2)
print 'Nodes expanded:', problem.expanded, '  Branching Factor: ', bf


def masterballStateWithSolutionAtDepth(depth, problem):
    state=[ [0, 1, 2, 3, 4, 5, 6, 7],
            [0, 1, 2, 3, 4, 5, 6, 7],
            [0, 1, 2, 3, 4, 5, 6, 7],
            [0, 1, 2, 3, 4, 5, 6, 7] ]
    actions=[]
    for i in range(0, 4):
        actions.append('Right- '+str(i))
    for i in range(4, 8):
        actions.append('Vertical '+str(i))
    state=state.__str__()
    act=''
    for i in range(0, depth):
        action = actions[randint(0,7)]
        while action==act:
            action = actions[randint(0,7)]
        state=problem.makeMove(state, action)
        act=action
    return state


labels = ['-Depth-','-IDS-', '-A*(h1)-', '-A*(h2)-']
labels = labels+['|']+labels
print '\n\n-------------- Generated Nodes -------------|-------- Effective Branching Factor --------'
print('{0:>10} {1:>10} {2:>10} {3:>10} {4:1} {5:>10} {6:>10} {7:>10} {8:>10}'.format(*labels))

for k in range(1,7):
    column1=[]
    column2=[]
    column1.append(k)
    column2.append(k)
    problem.setStartState(masterballStateWithSolutionAtDepth(k,problem))
    problem.reset()
    
    solveMasterBall(problem, iterativeDeepeningSearch(problem))
    bf=round(getBranchFactor(problem.expanded, k),2)
    column1.append(problem.expanded)
    column2.append(bf)
    problem.reset()
    
    solveMasterBall(problem, aStarSearch(problem, myHeuristic1))
    bf=round(getBranchFactor(problem.expanded, k),2)
    column1.append(problem.expanded)
    column2.append(bf)
    problem.reset()
    
    solveMasterBall(problem, aStarSearch(problem, myHeuristic2))
    bf=round(getBranchFactor(problem.expanded, k),2)
    column1.append(problem.expanded)
    column2.append(bf)
    
    row = column1+['|']+column2
    print('{0:10} {1:10} {2:10} {3:10} {4:1} {5:10} {6:10} {7:10} {8:10}'.format(*row))


-- IDS --
['Vertical 5', 'Right 1', 'Vertical 5', 'Vertical 1']
Nodes expanded: 450   Branching Factor:  4.32

-- A*(h1) --
['Vertical 5', 'Right 1', 'Vertical 1', 'Vertical 5']
Nodes expanded: 345   Branching Factor:  4.02

-- A*(h2) --
['Vertical 5', 'Right 1', 'Vertical 1', 'Vertical 5']
Nodes expanded: 402   Branching Factor:  4.19


-------------- Generated Nodes -------------|-------- Effective Branching Factor --------
   -Depth-      -IDS-   -A*(h1)-   -A*(h2)- |    -Depth-      -IDS-   -A*(h1)-   -A*(h2)-
         1          1          1          1 |          1        1.0        1.0        1.0
         2         14          3          3 |          2       3.27        1.3        1.3
         3         48         20         92 |          3       3.25       2.31       4.14
         4        345        308        406 |          4       4.02       3.89        4.2
         5       4325       1806       2019 |          5       5.11       4.25       4.35
         6      28569      152